<a href="https://colab.research.google.com/github/nangokosu/ebay_ml/blob/main/EfficientNet_Hard_Triple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.layers import *
import numpy as np
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import color,data,transform,filters,util,restoration
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 4.9MB/s 


In [ ]:
import efficientnet.keras as efn 

In [ ]:
def create_encoder(input_shape):
  inputs=Input(shape=input_shape)
  enet=efn.EfficientNetB1(include_top=False,weights=None,input_shape=(32,32,3),pooling=None)
  outputs=enet(inputs)
  outputs=GlobalAveragePooling2D()(outputs)
  model=keras.models.Model(inputs,outputs)
  return model

In [ ]:
import tensorflow.keras.backend as K
input=Input(shape=(32,32,3))
encoder=create_encoder((32,32,3))
encoded=encoder(input)
normalize_1=Lambda(lambda x: K.l2_normalize(x, axis=1))(encoded)
model=keras.models.Model(input,normalize_1)

In [ ]:
# Only run this if you are RETRAINING a model.
model=tf.keras.models.load_model("/content/drive/Shareddrives/eBay ML/EfficientNet_Triple_Hard",compile=True)

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
model_1 (Functional)         (None, 1280)              6575232   
_________________________________________________________________
lambda_1 (Lambda)            (None, 1280)              0         
Total params: 6,575,232
Trainable params: 6,513,184
Non-trainable params: 62,048
_________________________________________________________________


In [ ]:
#vector_image=create_encoder((32,32,3))

In [ ]:
loss=tfa.losses.TripletHardLoss(margin=1)
model.compile(loss=loss,optimizer="Adam")

In [ ]:
df = pd.read_pickle("/content/drive/Shareddrives/eBay ML/normalized_df_pairs_balanced.pkl")
length = df.shape[0]
df = df.iloc[2*length // 4:3*length // 4]

In [ ]:
image_processed=efn.preprocess_input(np.stack(df["image_1"]))

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe=LabelEncoder()
df["label_1_encoded"]=lbe.fit_transform(df["label_1"])

In [ ]:
lrplateau1=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.20,patience=8)
early_stop=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
modelCheckpoint=tf.keras.callbacks.ModelCheckpoint('/content/drive/Shareddrives/eBay ML/EfficientNet_Triple_Hard',save_best_only=True)

In [ ]:
model.fit(image_processed,df["label_1_encoded"],validation_split=0.10,epochs=40,batch_size=128,callbacks=[modelCheckpoint,early_stop,lrplateau1])

Epoch 1/40
694/694 [==============================] - 62s 62ms/step - loss: 0.1583 - val_loss: 0.5315
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_Triple_Hard/assets
Epoch 2/40
694/694 [==============================] - 41s 59ms/step - loss: 0.1604 - val_loss: 0.5333
Epoch 3/40
694/694 [==============================] - 41s 59ms/step - loss: 0.1565 - val_loss: 0.5302
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_Triple_Hard/assets
Epoch 4/40
694/694 [==============================] - 41s 59ms/step - loss: 0.1580 - val_loss: 0.5313
Epoch 5/40
694/694 [==============================] - 41s 59ms/step - loss: 0.1548 - val_loss: 0.5285
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_Triple_Hard/assets
Epoch 6/40
694/694 [==============================] - 41s 60ms/step - loss: 0.1554 - val_loss: 0.5271
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/Efficien

In [ ]:
import tensorflow as tf
tf.keras.models.save_model(model.layers[1],"/content/drive/Shareddrives/eBay ML/EfficientNet_TripleHard_encoder")

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_TripleHard_encoder/assets
